In [3]:
#-*- coding:utf-8 -*-
import pandas as pd
import numpy as np
from datetime import datetime
import time
#print time.localtime()
import json
import psycopg2
import redis

In [9]:
xls = pd.ExcelFile('OfferID_LIST_TAG.xlsx')
offer_table = xls.parse('Data_Offer')
offer_table.head()

,seg,UTID,offer1,offer2,offer3,offer4,offer5,offer6,offer7,offer8,offer9,offer10
0,小資女,1,OFF0012,OFF0057,OFF0055,OFF0032,OFF0047,OFF0019,OFF0008,OFF0007,OFF0031,OFF0059
1,時尚買客(百貨購物),2,OFF0057,OFF0055,OFF0062,OFF0019,OFF0012,OFF0008,OFF0047,OFF0007,OFF0011,OFF0031
2,聰明理財亨利族,3,OFF0031,OFF0058,OFF0062,OFF0056,OFF0008,OFF0061,OFF0021,OFF0011,OFF0007,OFF0060
3,世界任我行,4,OFF0021,OFF0007,OFF0011,OFF0061,OFF0056,OFF0008,OFF0059,OFF0047,OFF0057,OFF0055
4,量販大買家,5,OFF0059,OFF0055,OFF0007,OFF0047,OFF0057,OFF0058,OFF0062,OFF0021,OFF0012,OFF0008


In [10]:
offer_tag = list(offer_table)[2:]
offer_tag

[u'offer1',
 u'offer2',
 u'offer3',
 u'offer4',
 u'offer5',
 u'offer6',
 u'offer7',
 u'offer8',
 u'offer9',
 u'offer10']

In [12]:
str(offer_table.loc[0,:]['UTID'])

'1'

In [13]:
offer_table['offer_list'] = ""
offer_Json = []
for i in range(offer_table.shape[0]):
    temp_list = []
    temp_detail = {}
    temp_detail['tag_value'] = str(offer_table.loc[i,:]['UTID'])
    
    for column in offer_tag:
        temp_list.append(offer_table.loc[i,:][column])
    offer_table.loc[i,:]['offer_list'] = temp_list
    temp_detail['offer_list'] = temp_list
    offer_Json.append(temp_detail)

#seg = offer_table[['UTID','offer_list']]

/Users/llc/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [16]:
offer_table['offer_list']

0    
1    
2    
3    
4    
5    
6    
Name: offer_list, dtype: object

In [17]:
import psycopg2


# connect PostgreSQL
conn = psycopg2.connect(database = "MJ",
                        host = "localhost",
                        user = "llc",
                        password = "")
cur = conn.cursor()


In [20]:
for i in offer_Json:
    SQL = "INSERT INTO offer_tag_db(tag_info) VALUES('{}');".format(json.dumps(i))
    cur.execute(SQL)
    conn.commit()

In [22]:
def offer_tag_db_query(tag_value, sql_db):
    SQL = "SELECT tag_info FROM offer_tag_db WHERE tag_info->>'tag_value' = '{}';".format(tag_value)
    sql_db.execute(SQL)
    records = sql_db.fetchall()[0][0]
    return records

In [24]:
offer_tag_db_query(1,cur)

{u'offer_list': [u'OFF0012',
  u'OFF0057',
  u'OFF0055',
  u'OFF0032',
  u'OFF0047',
  u'OFF0019',
  u'OFF0008',
  u'OFF0007',
  u'OFF0031',
  u'OFF0059'],
 u'tag_value': u'1'}